In [ ]:
using Pkg
using Revise
Pkg.activate(joinpath(@__DIR__, "../../"))
using Globtim
using DynamicPolynomials, DataFrames
using GLMakie
using StaticArrays
# Constants and Parameters
const n, a, b = 3, 12, 100 
const scale_factor = a / b   # Scaling factor appears in `main_computation`
f = tref_3d # Objective function

  Activating 

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased.

In [ ]:
center = [0.0, 0.0, 0.0];
d = 14 # initial degree 
SMPL = 30 # Number of samples
TR = test_input(f, 
                dim = n,
                center=center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max =d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
# pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

Solve the system of partial derivatives using `Homotopy_COntinuation.jl`. 

In [ ]:
# df_cheb = solve_and_parse(pol_cheb, x, f, TR)
pts_cheb = solve_polynomial_system(x, TR.dim, d, pol_cheb.coeffs; basis=:chebyshev)
df_cheb = process_crit_pts(pts_cheb, f, TR)
sort!(df_cheb, :z, rev=false)

In [ ]:
grid = scale_factor * generate_grid(3, 100);  # 3D grid

In [ ]:
# fig_1 = create_level_set_visualization(f, grid, df_cheb, (-3.0, 6.0))

In [ ]:
fig_2 = create_level_set_animation(f, grid, df_cheb, (-3.0, 6.0))

In [ ]:
GLMakie.closeall() # Close all previous plots

In [ ]:
println(BigInt(2)^64)

## Function 1: add samples around one point

In [ ]:
function augment_grid_around_point(
    point::Vector{Float64},
    TR,
    N_sub::Int=5;
    cube_side_ratio::Float64=0.1,
    basis::Symbol=:legendre  # Changed default to legendre
)::Array{SVector{3,Float64},3}

    sample_range = TR.sample_range
    side_length = sample_range * cube_side_ratio
    half_side = side_length / 2

    # Using legendre (equally spaced) points
    x_coords = range(point[1] - half_side, point[1] + half_side, length=N_sub)
    y_coords = range(point[2] - half_side, point[2] + half_side, length=N_sub)
    z_coords = range(point[3] - half_side, point[3] + half_side, length=N_sub)

    [SVector{3,Float64}(x, y, z)
     for x in x_coords,
     y in y_coords,
     z in z_coords]
end

Is this the proper structure for merging the samples together? 

In [ ]:
gr_loc_1 = augment_grid_around_point(center, TR, 2)
gr_loc_2 = augment_grid_around_point([.1,.2,.3], TR, 2)
println(size(gr_loc_1))
println(size(gr_loc_2))
merged = cat(gr_loc_1, gr_loc_2, dims=3)
println(size(merged))

# Function 2: Create augmented grid using DataFrame points

In [ ]:
function create_augmented_grid(
    base_grid::Array{SVector{3,Float64},3},
    df_cheb::DataFrame;
    N_sub::Int=5,
    basis::Symbol=:legendre
)::Array{SVector{3,Float64},3}

    # Calculate new grid size
    current_size = size(base_grid, 1)  # Should be 40
    points_to_add = nrow(df_cheb) * N_sub^3
    new_side_length = ceil(Int, ∛(current_size^3 + points_to_add))

    # Create new grid array
    new_grid = Array{SVector{3,Float64},3}(undef, new_side_length, new_side_length, new_side_length)

    # Copy base grid
    for i in 1:size(base_grid, 1), j in 1:size(base_grid, 2), k in 1:size(base_grid, 3)
        new_grid[i, j, k] = base_grid[i, j, k]
    end

    # Add sub-sampled points in the remaining space
    current_idx = size(base_grid, 1) * size(base_grid, 2) * size(base_grid, 3)

    for row in eachrow(df_cheb)
        point = [row.x1, row.x2, row.x3]
        sub_grid = augment_grid_around_point(point, TR, N_sub; basis=basis)

        for point in vec(sub_grid)
            idx_i = (current_idx ÷ new_side_length^2) + 1
            idx_j = ((current_idx % new_side_length^2) ÷ new_side_length) + 1
            idx_k = (current_idx % new_side_length) + 1

            if idx_i <= new_side_length && idx_j <= new_side_length && idx_k <= new_side_length
                new_grid[idx_i, idx_j, idx_k] = point
                current_idx += 1
            end
        end
    end

    return new_grid
end

In [ ]:
# Create base grid with 40 points per dimension
GN = 50  # This will give us 40 points per dimension (GN + 1)
base_grid = generate_grid_small_n(3, GN, basis=:legendre)  # 40×40×40 grid
augmented_grid = create_augmented_grid(base_grid, df_cheb)
println(size(base_grid))
println(size(augmented_grid))

In [ ]:
function augment_samples_around_point(
    point::Vector{Float64},
    TR,
    N_sub::Int=5;
    cube_side_ratio::Float64=0.1
)::Vector{SVector{3,Float64}}
    sample_range = TR.sample_range
    side_length = sample_range * cube_side_ratio
    half_side = side_length / 2

    # Create local sampling points
    x_coords = range(point[1] - half_side, point[1] + half_side, length=N_sub)
    y_coords = range(point[2] - half_side, point[2] + half_side, length=N_sub)
    z_coords = range(point[3] - half_side, point[3] + half_side, length=N_sub)

    # Return as a flat vector of points
    return vec([SVector{3,Float64}(x, y, z) for x in x_coords, y in y_coords, z in z_coords])
end

function create_augmented_samples(
    base_points::Vector{SVector{3,Float64}},
    df_cheb::DataFrame;
    N_sub::Int=5
)::Vector{SVector{3,Float64}}
    # Initialize with base points
    augmented_points = Set{SVector{3,Float64}}()

    # Add base points
    for point in base_points
        push!(augmented_points, point)
    end

    # Add additional samples around critical points
    for row in eachrow(df_cheb)
        point = [row.x1, row.x2, row.x3]
        local_samples = augment_samples_around_point(point, TR, N_sub)
        for sample in local_samples
            push!(augmented_points, sample)
        end
    end

    return collect(augmented_points)
end

# Usage example:
# Convert your original grid to a vector if needed:
# base_points = vec(base_grid)
# augmented_points = create_augmented_samples(base_points, df_cheb)
# println("Original points: $(length(base_points))")
# println("Augmented points: $(length(augmented_points))")

In [ ]:
base_points = vec(base_grid)
augmented_points = create_augmented_samples(base_points, df_cheb)
println("Original points: $(length(base_points))")
println("Augmented points: $(length(augmented_points))") 

We are not correctly increasing the size of the sample grid. I thought we already had something efficient to augment the samples per level set, maybe the slider is much easier... In the end, I just want the Makie plot with the slider moving really slowly and a bit of rotation and output the result of that ---> 

In [ ]:
create_level_set_visualization(f, grid, df_cheb, (-3.0, 6.))

In [ ]:
create_level_set_visualization(f, augmented_grid, df_cheb, (-3.0, 6.0))

## Function 3: Produce Video 

In [ ]:
function make_level_sets_video(
    f,
    grid::Array{SVector{3,T},3},
    z_range::Tuple{Real,Real},
    output_path::String;
    fps::Int=30,
    duration::Float64=20.0,
    tolerance::Float64=0.1
) where {T<:AbstractFloat}

    n_frames = round(Int, fps * duration)
    level_sequence = range(z_range[1], z_range[2], length=n_frames)

    fig = Figure()
    ax = Axis3(fig[1, 1],
        title="Level Set Animation",
        xlabel="x₁",
        ylabel="x₂",
        zlabel="x₃")

    grid_points = vec(grid)
    x_range = extrema(p[1] for p in grid_points)
    y_range = extrema(p[2] for p in grid_points)
    z_range_grid = extrema(p[3] for p in grid_points)

    limits!(ax, x_range..., y_range..., z_range_grid...)

    values = reshape(map(f, grid_points), size(grid))
    level_points = Observable(Point3f[])
    point_alphas = Observable(Float32[])

    scatter!(ax, level_points,
        color=:blue,
        markersize=0.8,
        alpha=point_alphas)  # Changed from opacity to alpha

    function update_visualization(level::T) where {T<:AbstractFloat}
        all_points = Point3f[]
        all_alphas = Float32[]

        for (point, value) in zip(grid_points, vec(values))
            dist_from_level = abs(value - level)
            if dist_from_level ≤ tolerance
                alpha = 1.0 - (dist_from_level / tolerance)
                push!(all_points, Point3f(point...))
                push!(all_alphas, Float32(alpha))
            end
        end

        level_points[] = all_points
        point_alphas[] = all_alphas
    end

    record(fig, output_path, 1:n_frames; framerate=fps) do frame
        current_level = level_sequence[frame]
        update_visualization(current_level)

        ax.azimuth[] = 1.7π + 0.4 * sin(2π * frame / n_frames)
        ax.elevation[] = π / 4 + 0.3 * cos(2π * frame / n_frames)
    end
end

# Try it:
z_range = (-3.0, 6.0)
make_level_sets_video(tref_3d, augmented_grid, z_range, "level_sets_video.mp4"; tolerance=0.1)

In [ ]:
z_range = (-3.0, 6.0)
video_level_sets(tref_3d, augmented_grid, z_range, "level_sets_video.mp4"; tolerance=0.1)


We look at the level of the grid, we just sample more densely in `x, y, z` space around each of the critical points from the dataframe ? --> we reuse the grid to generate all the level sets. 

In [ ]:
# GLMakie.closeall()